# Curriculum Planning Assistant

This notebook implements an AI-powered curriculum planning system based on Wiggins and McTighe's 
Understanding by Design (UbD) framework (Bowen, 2017). UbD employs "backward design" - starting with 
desired results before planning assessments and learning activities.

Key Benefits of Backward Design (Wiggins & McTighe, 1998):
- Focuses on student learning outcomes rather than teaching activities
- Encourages intentional and explicit instruction
- Ensures activities align with learning goals
- Promotes student understanding rather than just content coverage

The system uses three stages of backward design:
1. Identify Desired Results
2. Determine Acceptable Evidence  
3. Plan Learning Experiences and Instruction

References:
Bowen, R. S. (2017). Understanding by Design. Vanderbilt University Center for Teaching. 
Retrieved from https://cft.vanderbilt.edu/understanding-by-design/

Wiggins, G., & McTighe, J. (1998). Understanding by Design. ASCD.

## Overview

We will be implementing a multi-crew, multi-agent orchestration that follows the UbD framework, by leveraging Crew.AI's sequential agent orchestration process. Here is a high level diagram of that flow.
![agent_flow](./assets/agentic_flow.png).

Throughout this implementation we've leveraged Amazon Nova for function calling, and Anthropic Claude for running the agents themselves. Feel free to update `llm_config.py` with other models you'd like to use.

## Setup

First, let's insetall and import the required libraries and define our helper functions:

In [ ]:
!pip install ipython==8.31.0 boto3==1.36.0 botocore==1.36.0 crewai==0.98.0 langchain-community==0.3.15

In [1]:
import json
from IPython.display import Markdown
from datetime import datetime
from info_collection_crew import InfoCollectionCrew
from curriculum_planning_crew import CurriculumPlanningCrew
import traceback
import crew_helpers

def load_student_info(json_file):
    """Load student information from a JSON file"""
    try:
        with open(json_file, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Error: Could not find {json_file}")
        raise
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {json_file}")
        raise

### Enable Nova and Claude Models in Bedrock
Be sure to request access to the following models, by following [these instructions](https://docs.aws.amazon.com/bedrock/latest/userguide/getting-started.html):
* anthropic.claude-3-5-haiku-20241022-v1
* us.amazon.nova-pro-v1:0


Once permissions have been granted you can begin using this notebook

### Get SearchApi Key
This example uses [Langchain's SearchApiAPIWrapper](https://python.langchain.com/docs/integrations/providers/searchapi/) to do both google searches for news and web content.
Alternatively, you can use the DuckDuckGo search to accomplish the same functionality. Note that this solution uses Google News as well.

### .env configuration

Next, make sure you've configured your AWS credentials chain. In this example we've leveraged a local AWS profile stored in a `.env` file. Feel free to copy `.env-copy` and use your own AWS profile in your own `.env`.
Be sure to also update `SEARCHAPI_API_KEY` to your search api key.

### Regional Availability
At the time of this writing, Nova is only available in `us-east-1`, so be sure to configure your environment to use this region. We've leveraged a cross-region inference profile in our solution to this end. See [llm_config.py](llm_config.py) `get_nova_pro()` method to see this.





## Load Student Information

We'll load the student information from a JSON file. 

This file should contain basic information about the student(s) and classroom context. It should also declare the `established_goals` of the curriculum, as the UbD method will workbackwards from this to design the learning plan.

You can find some examples in `/examples`.

Example JSON structure:
```json
{
    "location": "Springfield, IL",
    "number_students": "33",
    "age_range": "10-11",
    "venue": "classroom roughly 40X40 feet in size",
    "grade": "5th",
    "established_goals": "Students will demonstrate their ability to sequence major 20th century events using basic timeline concepts and vocabulary (past, present, future). Students will compare and contrast daily life between today and 100 years ago, focusing on tangible examples like transportation, communication, and school life. Students will recognize and explain the significance of key American symbols that represent our nation's values and history.",
    "special_needs": "One student who struggles with ADHD  regularly moves between moments of intense focus (particularly during hands-on or personally interesting activities) and periods of visible distraction when their attention drifts away from current tasks.  I have 1 staff support in my class on Wednesdays and Fridays from 1-3 PM."
}
```

In [ ]:
# Specify your JSON file path here
json_file_path = "examples/student_info.json"

# Load the base student information
base_student_info = load_student_info(json_file_path)

# Add current date to inputs
# local_timezone = datetime.now(timezone.utc).astimezone().tzinfo
inputs = {
    **base_student_info,
    "current_date": datetime.now().strftime("%A, %B %d, %Y")
}

print("Input information:", inputs)

## Information Collection Phase

The Information Collection Crew consists of three specialized agents:
1. Student Demographics Specialist
2. Classroom Environment Specialist
3. Community Liaison

This aligns with UbD Stage 1: Identifying Desired Results by understanding learner context.

These agents will gather relevant information about the students, classroom, and community context.

You can see the agent code in the `InfoCollectionCrew` class.

In [ ]:
try:
    info_crew = InfoCollectionCrew().crew()
    info_crew_result = info_crew.kickoff(inputs=inputs)
except Exception as e:
    print(f"Error during information collection: {str(e)}")
    traceback.print_exc()

Let's store our task outputs for later use as input to our curriculum development.

In [ ]:
collected_info = crew_helpers.task_summary(crew_output=info_crew_result)
Markdown(collected_info)

## Curriculum Planning Phase

The Curriculum Planning Crew implements the three UbD stages (Wiggins & McTighe, 1998):

**Stage 1 - Identify Desired Results:**
- What should students know and be able to do?
- What enduring understandings should students retain?

**Stage 2 - Determine Acceptable Evidence:**
- How will we know if students have achieved the desired results?
- What evidence will show understanding?

**Stage 3 - Plan Learning Experiences:**
- What knowledge and skills do students need?
- What activities will help students learn?
- What resources and teaching methods should be used?

You can see the agent code in the `CurriculumPlanningCrew` class.

In [ ]:
try:
    curriculum_planning_crew = CurriculumPlanningCrew().crew()
    curriculum_planning_crew_result = curriculum_planning_crew.kickoff(inputs={**inputs, "background": collected_info})

    # print("\nCurriculum Planning Results:")
    # Markdown(curriculum_planning_crew_result.raw)
except Exception as e:
    print(f"Error during curriculum planning: {str(e)}")
    traceback.print_exc()

Now that our crew has done its job, lets come back to our original established goals for this curriculum.

In [ ]:
Markdown(f"# Established Goals\n #### {base_student_info['established_goals']}")

Now lets take a look at our full output from our crew, but combining all of the task outputs.

In [ ]:
curriculum_planning_crew_result_md = crew_helpers.task_summary(crew_output=curriculum_planning_crew_result)

Markdown(f"# Established Goals\n\n{curriculum_planning_crew_result_md}")

## Results

The above cells will generate:

**Stage 1 Results:**
- Transfer goals
- Enduring understandings 
- Essential questions
- Knowledge and skills

**Stage 2 Results:**
- Assessment evidence
- Performance tasks
- Other evidence

**Stage 3 Results:**
- Learning activities
- Instructional approaches
- Resources needed

This comprehensive plan follows the UbD framework to create meaningful, assessment-driven instruction
focused on student understanding.

You can use these results to implement an effective, contextualized curriculum for your classroom.

Find each of the task outputs in their respective log folders in `/logs/tasks/outputs/`.


## Next Steps

Take a deeper dive into how you can make this workflow work best for your students. 

Are there other tools your organization uses to ensure students are measurably improving their learning?

What datasources could this workflow utilize to be improved?


**Implementation**
* Review and customize the generated curriculum plan for their specific needs
* Break down the curriculum into detailed weekly/daily lesson plans
* Create specific teaching materials and handouts based on the learning activities
* Set up assessment rubrics using the evaluation criteria provided
* Develop a timeline for implementing different curriculum components

**Technical Enhancement**
* Add more specialized agents to handle specific aspects like special education needs
* Integrate with existing Learning Management Systems (LMS)
* Create a web interface for easier input of student/classroom information
* Add functionality to export curriculum plans in different formats (PDF, DOC, etc.)

**Extension Projects**
* Develop complementary tools for resource gathering
* Build assessment generators aligned with curriculum goals
* Design parent communication templates
* Create student portfolio systems aligned with curriculum objectives
